In [8]:
import sys
import numpy as np
import pandas as pd
import cv2
from pycocotools.coco import COCO
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage

class Process(object):
    numKeypoint = 17
    keypointsName = ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 
                    'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 
                    'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 
                    'right_knee', 'left_ankle', 'right_ankle']

    skeleton = [[15, 13], [13, 11], [16, 14], 
                [14, 12], [11, 12], [5, 11], 
                [6, 12], [5, 6], [5, 7], [6, 8], 
                [7, 9], [8, 10], [1, 2], [0, 1], 
                [0, 2], [1, 3], [2, 4], [3, 5], [4, 6]]
    
    trainAnnoPath = 'COCO/annotations/person_keypoints_train2017.json'
    valAnnoPath = 'COCO/annotations/person_keypoints_val2017.json'
    
    def loadData(self, setType):
        if setType == 'train':
            coco = COCO(self.trainAnnoPath)
        elif setType == 'val':
            coco = COCO(self.valAnnoPath)
            
        Data = []
        for aid in coco.anns.keys():
            ann =  coco.anns[aid]
            keypoints = ann['keypoints']
            
            if  keypoints.count(0) == 0: 
                imageName = setType + '2017/' + coco.imgs[ann['image_id']]['file_name']

                if (ann['image_id'] not in coco.imgs) or ann['iscrowd'] or (np.sum(keypoints[2::3]) == 0) or (ann['num_keypoints'] == 0):
                    continue

                data = dict(image_id = ann['image_id'], imgpath = imageName, bbox=ann['bbox'], keypoints=keypoints)
                Data.append(data)
            else:
                continue
            
        return Data
    
    def imagesToNdarry(self, data): 
        imgPath = []
        i = 0
        for i in range(len(data)):
            img = cv2.imread('COCO/' + data[i]['imgpath'])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgPath.append(img)
            
        return np.array(imgPath)
    
    def splitKeypoints(self, data):
        kpsList = []
        visibiltyList = []
        for ann in data:
            keypoints = np.array(ann['keypoints']).astype(int)
            keypoints =  keypoints.reshape(17,3)
            keypoints = self.reshapeKeypoints(keypoints)
            kpsList.append(keypoints['kps'])
            visibiltyList.append(keypoints['visibility'])

        return kpsList, visibiltyList
    
    def reshapeKeypoints(self, keypoints):
        keypoint = []
        visibility = []

        for kps in keypoints: 
            x,y,v = kps 
            keypoint.append(Keypoint(x,y))
            visibility.append(v)
            
        res = dict()
        res['kps'] = keypoint; res['visibility'] = visibility
        return res
    
#     def augmentKeypoints(self, keypoints, shape):
#         augKpsList = []
#         for kps in keypoints:
#             augKps = self.augmentation.augment_keypoints(KeypointsOnImage(kps, shape))
#             augKpsList.append(augKps)
#         return augKpsList

    def augmentationOnAll(self, images, kps):
        seq = iaa.Sequential([
            iaa.Resize(112),            
        ])

        imgAugList = []
        kpsAugList = []
        
        i = 0 
        for i in range(len(images)):
            imgAug, kpsAug = seq(image=images[i], keypoints=KeypointsOnImage(kps[i],shape=images[i].shape))
            imgAugList.append(imgAug)
            kpsAugList.append(kpsAug)
            
        return np.array(imgAugList), kpsAugList

    def mergeKpsVisibility(self, augKps, visibility, kpShape, MergeShape):
        kpsList = [] 
        i = 0
        for i in range(len(augKps)):
            for keypoint in augKps[i]:
                x,y = keypoint.x, keypoint.y
                kpsList.append(x)
                kpsList.append(y)
        kpsList = np.array(kpsList).reshape(kpShape)
        merge = np.dstack((kpsList, visibility))
        merge = merge.reshape(MergeShape)
        return merge
    
    def formatToDataframe(self, anno, keypoints):
        col_name  = ['nose_x', 'nose_y', 'nose_v',
                     'left_eye_x','left_eye_y', 'left_eye_v', 
                     'right_eye_x', 'right_eye_y', 'right_eye_v',
                     'left_ear_x', 'left_ear_y', 'left_ear_v',
                     'right_ear_x', 'right_ear_y', 'right_ear_v',
                    'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_v',
                     'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_v',
                     'left_elbow_x', 'left_elbow_y', 'left_elbow_v',
                     'right_elbow_x', 'right_elbow_y', 'right_elbow_v', 
                    'left_wrist_x', 'left_wrist_y', 'left_wrist_v',
                     'right_wrist_x', 'right_wrist_y', 'right_wrist_v',
                     'left_hip_x', 'left_hip_y', 'left_hip_v', 
                     'right_hip_x', 'right_hip_y', 'right_hip_v',
                     'left_knee_x', 'left_knee_y', 'left_knee_v',
                    'right_knee_x', 'right_knee_y', 'right_knee_v', 
                     'left_ankle_x', 'left_ankle_y', 'left_ankle_v', 
                     'right_ankle_x', 'right_ankle_y', 'right_ankle_v',
                     'image_path'  
                    ]
        data = []
        
        i = 0
        for arr in keypoints:
            cur_row = []
            kps = np.array(arr).astype(int)
            for key in kps:
                cur_row.append(key)
            cur_row.append('COCO/aug-' + anno[i]['imgpath'])
            data.append(cur_row)
        
        return pd.DataFrame(data=data, columns=col_name)

processing = Process()

In [9]:
from sklearn.model_selection import train_test_split as tts
import numpy as np

class Setup(object):
    setType = ''
    batchsize = 0
    fetchedData = []
    
    def __init__ (self, setType, batchSize):
        self.setType = setType
        self.batchsize = batchSize
        
    def getTrainingSet(self):
        batch = processing.loadData('train')
        batch = batch[:self.batchsize]
        
        images = processing.imagesToNdarry(batch)
        test = images[0]
        kpsList, visibilityList = processing.splitKeypoints(batch)
        
        x, kps = processing.augmentationOnAll(images, kpsList)

        Y = processing.mergeKpsVisibility(kps, visibilityList, (self.batchsize, 17,2), (self.batchsize, 51))
        images = [] 
        
        return Y, batch

In [10]:
setup = Setup('train', 8000)

In [11]:
Y, batch = setup.getTrainingSet()

loading annotations into memory...
Done (t=11.06s)
creating index...
index created!


In [12]:
df = processing.formatToDataframe(batch, Y)

In [13]:
df.head()

,nose_x,nose_y,nose_v,left_eye_x,left_eye_y,left_eye_v,right_eye_x,right_eye_y,right_eye_v,left_ear_x,...,right_knee_x,right_knee_y,right_knee_v,left_ankle_x,left_ankle_y,left_ankle_v,right_ankle_x,right_ankle_y,right_ankle_v,image_path
0,91,37,2,91,34,2,90,35,2,93,...,86,63,2,93,86,2,87,73,2,COCO/aug-train2017/000000528962.jpg
1,70,46,2,71,45,2,69,45,2,71,...,68,70,2,72,81,2,68,81,2,COCO/aug-train2017/000000528962.jpg
2,56,21,2,56,21,2,55,20,2,57,...,54,51,1,57,60,2,52,61,2,COCO/aug-train2017/000000528962.jpg
3,56,5,2,58,4,2,55,4,2,60,...,50,28,2,55,58,1,47,44,2,COCO/aug-train2017/000000528962.jpg
4,56,21,2,58,18,2,52,18,2,62,...,44,74,1,60,88,1,45,91,1,COCO/aug-train2017/000000528962.jpg


In [14]:
df.to_csv('augDATA.csv', index=False)